In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Load data

In [11]:
buy = pd.read_csv('./buy_data.csv') # 구매이력
history = pd.read_csv('./history_data.csv') # 시청 시작 데이터
watch = pd.read_csv('./watch_e_data.csv') # 시청 종료 데이터
meta = pd.read_csv('./meta_data.csv') # 콘텐츠 일반 메타
meta_plus = pd.read_csv('./meta_data_plus.csv') # 콘텐츠 확장 정보
profile = pd.read_csv('./profile_data.csv') # 프로필 정보
search = pd.read_csv('./search_data.csv') # 검색을 통한 시청 데이터
sample = pd.read_csv('./sample_submission.csv')


/var/folders/fb/fgwpc_kd3dxdcfsnm_9w3wv40000gn/T/ipykernel_75742/3129629035.py:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv('./meta_data.csv') # 콘텐츠 일반 메타


제공된 학습 데이터를 통하여 사용자가 이후 시청할 콘텐츠를 추천하는 것이 최종 목표

In [12]:
sample

,profile_id,predicted_list
0,3,[0]
1,5,[0]
2,7,[0]
3,12,[0]
4,16,[0]
...,...,...
8306,33022,[0]
8307,33023,[0]
8308,33026,[0]
8309,33027,[0]


CF : ALS를 사람 그룹별로 쪼개서 ALS 를 돌려버린다

In [78]:
meta

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7
0,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
1,750,우리는 친구,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
2,2131,타요의 첫 운행,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
3,2625,길 잃은 타요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
4,2594,새내기 꼬마 버스의 하루,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42597,39873,로티프렌즈와 색칠놀이! - 그리피 ＆ 사탕 바구니,로티프렌즈 미술놀이,키즈,놀이교실,NaN,한국,477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42598,39874,로티프렌즈와 색칠놀이! - 베블리 ＆ 꽃,로티프렌즈 미술놀이,키즈,놀이교실,NaN,한국,466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42599,4779,손가락을 빨게 돼요,4-5세,키즈,책,NaN,한국,293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42600,4779,손가락을 빨게 돼요,아이맘콕,키즈,책,NaN,한국,293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
history

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N
...,...,...,...,...,...,...,...,...
1005646,33032,20220427155091,20220427155668,MKID003,381,NaN,Y,N
1005647,33032,20220427155091,20220427155680,MKID003,381,NaN,Y,N
1005648,33032,20220427155839,20220427155810,MKID003,125,NaN,Y,N
1005649,33032,20220427155706,20220427155838,MKID003,125,NaN,Y,N


In [74]:
history.isnull().sum()

profile_id              0
ss_id                   0
log_time                0
act_target_dtl          0
album_id                0
payment            932165
continuous_play         0
short_trailer           0
dtype: int64

In [79]:
profile

,profile_id,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,3,F,5,P02,P04,P07,K01,K03,K04
1,5,M,5,P07,P08,P06,K05,K08,K09
2,7,F,9,P05,P03,NaN,K06,K04,NaN
3,12,M,6,P03,P06,P02,K09,K07,K03
4,16,F,12,P03,P06,P01,K01,K06,K04
...,...,...,...,...,...,...,...,...,...
8306,33022,M,1,P04,NaN,NaN,K04,K08,NaN
8307,33023,M,5,P06,P03,P07,K08,K04,K05
8308,33026,F,8,P01,P03,P08,K05,K09,K06
8309,33027,F,4,P04,P05,P06,K03,K01,K05


In [85]:
meta['genre_large'].value_counts()

키즈     40224
애니      1991
방송       295
다큐        58
라이프       21
영화        13
Name: genre_large, dtype: int64

In [17]:
watch

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play
0,3,20220301115653,20220301115805,MKID049,15,46,46,1
1,3,20220301115653,20220301115954,MKID049,16,104,105,1
2,3,20220301115653,20220301120114,MKID049,17,76,76,1
3,3,20220301115653,20220301120226,MKID049,18,67,68,1
4,3,20220301115653,20220301120400,MKID049,19,90,90,0
...,...,...,...,...,...,...,...,...
892789,33032,20220427155091,20220427155653,MKID049,381,463,464,1
892790,33032,20220427155091,20220427155694,MKID049,381,462,463,1
892791,33032,20220427155839,20220427155826,MKID049,125,10,520,0
892792,33032,20220427155706,20220427155836,MKID049,125,6,521,0


In [69]:
len(set(history['profile_id']) - set(watch['profile_id']))

653

In [68]:
set(watch['profile_id'])-set(history['profile_id'])

set()

In [73]:
set(sample['profile_id']) - set(history['profile_id'])

set()

CBF : meta_plus에 키워드 다 추출되어있는 상태

In [28]:
watch.profile_id.nunique()

7658

In [27]:
meta_plus.sort_values(by='album_id')
meta_plus.album_id.nunique()

39875

In [34]:
# 전부다 들어가 있다.
watch[~watch.profile_id.isin(meta_plus.album_id)]

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play


In [66]:
keyword = dict()
for row in meta_plus.itertuples():
    if keyword[row.keyword_name]:
        keyword[row.keyword_name].append(row.keyword_type)
    else :
        keyword[row.keyword_name] = [row.keyword_type]
#     keyword[meta_plus['keyword_type']] += meta_plus['keyword_name']

KeyError: '시리즈'

In [58]:
meta_plus

,album_id,keyword_type,keyword_name,keyword_value
0,749,05010101,시리즈,0
1,749,05010302,EBS,0
2,749,05010401,3세,0
3,749,05010402,4세,0
4,749,05010407,1세,0
...,...,...,...,...
767943,11629,05040104,신나는,0
767944,11629,0504010b,따라하고 싶은,0
767945,11629,05070e01,인체,0
767946,11629,05080401,캐릭터 친구들,0


In [53]:
meta_plus['keyword_name'].value_counts()

시리즈      32644
엔터       30040
사회정서     26135
언어1      23611
TV 방영    16553
         ...  
한부모가족        1
태교이론         1
보상방법         1
용감함 4        1
수상가옥         1
Name: keyword_name, Length: 1062, dtype: int64

In [54]:
meta_plus['keyword_type'].value_counts()

05010101    32644
05020203    30040
05090806    26135
05090802    23611
05010601    16553
            ...  
05030808        1
05010e03        1
05010910        1
0509071c        1
05040228        1
Name: keyword_type, Length: 1101, dtype: int64

In [42]:
kname = meta_plus['keyword_name'].value_counts().reset_index()
ktype = meta_plus['keyword_type'].value_counts().reset_index()

In [43]:
ktype['keyword_type'][0]

32644

In [49]:
index = []
for i in range(len(kname)):
    if ktype['keyword_type'][i] != kname['keyword_name'][i]:
        index.append(i)

    

In [50]:
index

[7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 69,
 70,
 71,
 72,
 81,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 129,
 130,
 133,
 134,
 135,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 